In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, Adagrad

In [ ]:
import PGNN_source as pg
import NNNN_source as nn

In [ ]:
def compute_test_rmse(y_test, mu_test, offset_test = None):
    if offset_test is None:
        return np.sqrt(np.mean((y_test - mu_test)**2))
    else:
        return np.sqrt(np.mean(((y_test - mu_test)*offset_test)**2))
    
def compute_test_rmsr(y_test, mu_test, offset_test = None):
    if offset_test is None:
        return np.sqrt(np.mean(((y_test - mu_test)**2)/mu_test))
    else:
        return np.sqrt(np.mean(((y_test - mu_test)**2)*offset_test/mu_test))

In [ ]:
dir_name = os.getcwd()
data_type_list = ['1000-10-0-fixed', '1000-10-0.5-gamma', '1000-10-1-gamma', '1000-10-0.5-normal', '1000-10-1-normal']

n_simul = 100
phi_init, lam_init = 0.8, 0.8

lr = 0.005
optimizer = Adam(learning_rate=lr)

patience, pretrain, moments_epochs, max_epochs = 50, 50, 50, 500
callbacks = [EarlyStopping(monitor='val_loss', patience=patience)]

nodes = [10, 10, 10]
activation = 'leaky_relu'

# P-NN

In [ ]:
def make_mean_model_PX(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        expxb = Dense(1, activation='exponential')(m)
    else: expxb = Dense(1, activation='exponential')(input_X)   
    mean_model = Model(inputs=[input_X], outputs=[expxb])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_PX(nodes, activation)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.Poisson())
        start_time = time.time()
        M_history = M.fit([X_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid], y_valid))
        mu_test = np.float32(M([X_test])).T
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')

    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# PF-NN

In [ ]:
def make_mean_model_PF(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        expxb = Dense(1, activation='exponential')(m)
    else: expxb = Dense(1, activation='exponential')(input_X)
    expzv = Dense(1, activation='exponential', use_bias=False)(input_Z)        
    mean_model = Model(inputs=[input_X, input_Z], outputs=[expxb*expzv])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_PF(nodes, activation)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.Poisson())
        start_time = time.time()
        M_history = M.fit([X_train, Z_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid, Z_valid], y_valid))
        mu_test = np.float32(M([X_test, Z_test])).T
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')
    
    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# PG-NN

In [ ]:
def make_mean_model_PG(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        xb = Dense(1, activation='linear')(m)
    else: xb = Dense(1, activation='linear')(input_X)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb, zv])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_PG(nodes, activation)
        start_time = time.time()
        res = pg.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             pg.pg_hlik_loss, optimizer, lam_init, batch_ratio, patience, pretrain, max_epochs, moments_epochs)
        mu_test = np.exp(np.sum(M([X_test, Z_test]), axis=0).T)
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')
    
    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# N-NN

In [ ]:
def make_mean_model_NX(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        xb = Dense(1, activation='linear')(m)
    else: xb = Dense(1, activation='linear')(input_X)   
    mean_model = Model(inputs=[input_X], outputs=[xb])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_NX(nodes, activation)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
        start_time = time.time()
        M_history = M.fit([X_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid], y_valid))
        mu_test = np.float32(M([X_test])).T
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')
    
    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# NF-NN

In [ ]:
def make_mean_model_NF(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        xb = Dense(1, activation='linear')(m)
    else: xb = Dense(1, activation='linear')(input_X)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb+zv])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_NF(nodes, activation)
        M.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
        start_time = time.time()
        M_history = M.fit([X_train, Z_train], y_train, epochs=max_epochs, batch_size=batch_size, verbose=0, 
            callbacks=callbacks, validation_data=([X_valid, Z_valid], y_valid))
        mu_test = np.float32(M([X_test, Z_test])).T
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')
    
    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# NN-NN

In [ ]:
def make_mean_model_NN(nodes, activation):    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')
    if len(nodes)!=0:
        m  = Dense(nodes[0], activation=activation)(input_X)
        for i in range(1,len(nodes)):
            m  = Dense(nodes[i], activation=activation)(m)
        xb = Dense(1, activation='linear')(m)
    else: xb = Dense(1, activation='linear')(input_X)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb, zv])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_NN(nodes, activation)
        start_time = time.time()
        res = nn.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             nn.nn_hlik_loss, optimizer, phi_init, lam_init, batch_ratio, patience, pretrain, max_epochs, moments_epochs)
        mu_test = np.sum(M([X_test, Z_test]), axis=0).T
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')
    
    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))

# PG-GLM

In [ ]:
def make_mean_model_HL():    
    input_X = Input(shape=(np.shape(X_train)[1],), dtype='float32')
    input_Z = Input(shape=(np.shape(Z_train)[1],), dtype='float32')    
    xb = Dense(1, activation='linear')(input_X)
    zv = Dense(1, activation='linear', use_bias=False)(input_Z)
    mean_model = Model(inputs=[input_X, input_Z], outputs=[xb, zv])
    return mean_model

In [ ]:
model_rmse, model_rmsr, model_time = [np.zeros((n_simul, len(data_type_list))) for _ in range(3)]
for data_type in data_type_list:     

    colnum = data_type_list.index(data_type)        
    n_sub, n_num, lam, rand_dist = data_type.split('-')
    n_sub, n_num, lam = int(n_sub), int(n_num), float(lam)
    n_num_train, n_num_valid, n_num_test = int(n_num*0.6), int(n_num*0.2), int(n_num*0.2)
    N_train, N_valid, N_test = n_sub * np.array([n_num_train, n_num_valid, n_num_test])
    mu_pred = pd.DataFrame(np.zeros((N_test,n_simul)), columns=[str(x) for x in range(n_simul)], dtype=np.float32)

    for simul_num in tqdm(range(n_simul)):

        file_name = dir_name + 'simul-data-' + data_type + '-' + str(simul_num)
        data = pd.read_csv(file_name+'.csv')
        data_train = data[data['num'].isin(range(n_num_train))]
        data_valid = data[data['num'].isin(range(n_num_train, n_num - n_num_test))]
        data_test = data[-data['num'].isin(range(n_num - n_num_test))]
        subset_names = ['_train', '_valid', '_test']
        for subset in subset_names:
            exec('temp_data = data'+subset)
            exec('X'+subset+'= np.array(temp_data[["x"+str(i) for i in range(5)]], dtype=np.float32)')
            exec('y'+subset+'= np.array(temp_data["y"], dtype=np.float32)')        
            exec('z'+subset+'= np.array(temp_data["sub"].astype("int32"))')
            exec('Z'+subset+'= np.eye(n_sub)[z'+subset+'].astype("float32")')
            exec('N'+subset+'= n_sub*n_num'+subset)
        batch_size, batch_ratio = N_train, 1.            
        pg.seed_everything()
        train_batch = tf.data.Dataset.from_tensor_slices((X_train, Z_train, y_train)).shuffle(N_train).batch(batch_size)    

        K.clear_session(); pg.seed_everything()
        M = make_mean_model_HL()
        start_time = time.time()
        res = pg.train_model(M, train_batch, [X_train, Z_train, y_train], [X_valid, Z_valid, y_valid],
             pg.pg_hlik_loss, optimizer, lam_init, batch_ratio, patience, pretrain, max_epochs, moments_epochs)
        mu_test = np.exp(np.sum(M([X_test, Z_test]), axis=0).T)
        timesec = time.time() - start_time

        mu_pred[str(simul_num)] = mu_test.T        
        model_rmse[simul_num, colnum] = compute_test_rmse(y_test, mu_test)
        model_rmsr[simul_num, colnum] = compute_test_rmsr(y_test, mu_test)
        model_time[simul_num, colnum] = timesec

    mu_pred.to_csv(dir_name+data_type+'-'+model_name+'-pred.csv')

    print(data_type)
    print('time: ', np.round(np.mean(model_time, axis=0),3))
    print('rmse: ', np.round(np.mean(model_rmse, axis=0),3))
    print('rmsr: ', np.round(np.mean(model_rmsr, axis=0),3))